[*The Museum of Modern Art (MoMA) Collection*](https://github.com/MuseumofModernArt/collection)

description

motivation

questions

*explores and presents information about your data*

* This can include summary information about the data, summary statistics where appropriate, an identification of where there may be missing/incorrect/outlier data, and data preprocessing
* It should include at least 4 exploratory data visualizations

In [ ]:
# begin of analysis

# visualization 1

# visualization 2

# visualization 3

# visualization 4